<a href="https://colab.research.google.com/github/nataliesioo/NLP-NgramAndWSD/blob/master/NLP_NGram%26WSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**N-Gram Language Modeling(통계적 기반)**
* N-gram: n개의 단어가 연속적으로 등장한다
* LM: 언어의 시퀀스?를 학습한다


통계적 기반


In [0]:
# 4개 단어 연속적으로 나올 가능성
# 1의 가능성 + 1나오면 2 나올 가능성 + 1,2나올때 3 나올 사능성....... 다만 너무 작은 값이 나올거라
# 아래와 같은 방법으로 계산함

p(1,2,3,4) = p(1) * p(2|1) * p(3|1,2) * p(4|1,2,3) # 이 방법으로 하면 로그 취해도 너무 작아  
p(1,2,3,4) = p(1) * p(2|1) * p(3|2) * p(4|3) #이거씀

**English**

In [0]:
# package
import nltk
from nltk.util import ngrams
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist

In [4]:
# load dataset (corpus 다운받음)
nltk.download('punkt')
nltk.download('brown')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [5]:
# n-gram
bigrams = []
for tokens in nltk.corpus.brown.sents(): #corpus안에 있는 sentence bigram에 저장
  print(tokens) #띄어쓰기 단위

  bigrams += ngrams(tokens,2) #bigram = 2 , trigram = 3
  print(bigrams) #바이그램 형태로 저장된것 출력
  break

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
[('The', 'Fulton'), ('Fulton', 'County'), ('County', 'Grand'), ('Grand', 'Jury'), ('Jury', 'said'), ('said', 'Friday'), ('Friday', 'an'), ('an', 'investigation'), ('investigation', 'of'), ('of', "Atlanta's"), ("Atlanta's", 'recent'), ('recent', 'primary'), ('primary', 'election'), ('election', 'produced'), ('produced', '``'), ('``', 'no'), ('no', 'evidence'), ('evidence', "''"), ("''", 'that'), ('that', 'any'), ('any', 'irregularities'), ('irregularities', 'took'), ('took', 'place'), ('place', '.')]


In [7]:
# n-gram
bigrams = []
for tokens in nltk.corpus.brown.sents(): #corpus안에 있는 sentence bigram에 저장
  bigrams += ngrams(tokens,2) #bigram = 2 , trigram = 3
print(bigrams[:10])


[('The', 'Fulton'), ('Fulton', 'County'), ('County', 'Grand'), ('Grand', 'Jury'), ('Jury', 'said'), ('said', 'Friday'), ('Friday', 'an'), ('an', 'investigation'), ('investigation', 'of'), ('of', "Atlanta's")]


In [8]:
#빈도 분포? sentence generation
cfd = ConditionalFreqDist(bigrams)
cfd['the'].most_common(5) # the 뒤에 가장 많이 나온 단어 5개. 카운트수 나옴. 

[('same', 597), ('first', 539), ('other', 384), ('most', 376), ('United', 374)]

In [9]:
# sentence prob
cpd = ConditionalProbDist(cfd,MLEProbDist)
cpd['the'].prob('most') # the 뒤에 most 나온 의 prob. 확률 나옴


0.005996332030938522

**Korean**

In [10]:
#package
!pip install konlpy
import konlpy
from konlpy.tag import Kkma

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 12.7MB/s 
     |████████████████████████████████| 2.9MB 49.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [0]:
#load dataset (corpus)
with konlpy.corpus.kolaw.open('constitution.txt') as f:
  corpus = f.read()

In [12]:
# tokenization
# 띄어쓰기 단위로 나눠지지 않고 형태소로 나눠져야함. 아님 양이 넘 많음
kkma = Kkma() #kkma 형태소 분석기. 시간 오래 걸리지만 정확도 높음
tokens = kkma.morphs(corpus) 
print(tokens[:10])

['대한민국', '헌법', '유구', '하', 'ㄴ', '역사', '와', '전통', '에', '빛나']


In [0]:
# n-gram
bigrams = ngrams(tokens,2) # bigram에 저장
cfd = ConditionalFreqDist(bigrams)
cfd['대한민국'].most_common(5) # 대만민국 뒤에 나올 것


[('의', 5), ('은', 3), ('헌법', 1), ('임시', 1), ('영역', 1)]

In [0]:
# sentence prob
cpd = ConditionalProbDist(cfd, MLEProbDist) # MLEProbDist 11개중 5개
cpd['대한민국'].prob('의') # 대만민국 뒤에 의 나올 가능성


0.45454545454545453

**Word-Sense Disambiguation** (중의성 해소)
* Lesk alg : 통계적으로 중의성 해소 
  * sense: 워드의 대한 사전적 정의 
    * 하나 선택해 best-sense로 선출
  * max overlap : 최대한 얼마나 겁치는지
  * sense가지는 signature,context 얼마나 겁치는지 찾아내고
  * overlap > max-overlap: 取代舊max&best-sense
  * returen best sense + word

In [14]:
# packages
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet
nltk.download ('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [15]:
# WSD using LESK alg
sent = ['It','is','not','because','I','was','trying','to','save','money']
lesk(sent,'save','v') # sent안에 등장한 save란 단어 = 5번째 의미다.


Synset('save.v.05')

In [0]:
# 5번째 단어가 무슨 의민지 몰라서 알기 위해
for synset in wordnet.synsets('save'):
  print(synset,synset.definition())

# 5번째 의미: Synset('save.v.05') accumulate money for future use

Synset('save.n.01') (sports) the act of preventing the opposition from scoring
Synset('salvage.v.01') save from ruin, destruction, or harm
Synset('save.v.02') to keep up and reserve for personal or special use
Synset('save.v.03') bring into safety
Synset('save.v.04') spend less; buy at a reduced price
Synset('save.v.05') accumulate money for future use
Synset('save.v.06') make unnecessary an expenditure or effort
Synset('deliver.v.08') save from sins
Synset('spare.v.01') refrain from harming
Synset('save.v.09') spend sparingly, avoid the waste of
Synset('keep_open.v.01') retain rights to
Synset('write.v.08') record data on a computer
